In [1]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf 

In [3]:
url = "https://raw.githubusercontent.com/Agablue-red/Machine-Learning/master/data/CONVICTIONLISTTOPN_BSLD-408.csv"

In [4]:
df = pd.read_csv(url, index_col=False, names=['info', 'date', 'symbol', 'symbol2', 'sector', 'number', 'score'])
df

,info,date,symbol,symbol2,sector,number,score
0,10:01:54.481 77425 [77425-thread-2] INFO a.s....,2004-02-11,SU,SU,Energy Minerals,GN63J3-R,0.953727
1,10:01:54.481 77425 [77425-thread-2] INFO a.s....,2004-02-11,GGG,GGG,Producer Manufacturing,H5490W-R,0.952753
2,10:01:54.481 77425 [77425-thread-2] INFO a.s....,2004-02-11,WGR,WGR,Energy Minerals,V0622Q-R,0.947634
3,10:01:54.481 77425 [77425-thread-2] INFO a.s....,2004-02-11,CWT,CWT,Utilities,GSWXLY-R,0.934181
4,10:01:54.481 77425 [77425-thread-2] INFO a.s....,2004-02-11,BLL,BLL,Process Industries,VFT0VQ-R,0.922862
...,...,...,...,...,...,...,...
37355,10:27:03.049 77425 [77425-thread-2] INFO a.s....,2022-02-09,PEP,PEP,Consumer Non-Durables,PPCTFP-R,0.701507
37356,10:27:03.049 77425 [77425-thread-2] INFO a.s....,2022-02-09,SSNC,SSNC,Technology Services,G92RX2-R,0.701123
37357,10:27:03.049 77425 [77425-thread-2] INFO a.s....,2022-02-09,GEF,GEF,Process Industries,MPX0N4-R,0.697954
37358,10:27:03.049 77425 [77425-thread-2] INFO a.s....,2022-02-09,DPZ,DPZ,Consumer Services,F05QG0-R,0.697741


In [5]:
#usuniecie niepotrzebnych kolumn
df = df.drop(['info','symbol2','number'], axis=1)

In [6]:
#konwertuj wartosci kolumny date na typ DATETIME
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [7]:
df.head(5)

,symbol,sector,score
date,,,
2004-02-11,SU,Energy Minerals,0.953727
2004-02-11,GGG,Producer Manufacturing,0.952753
2004-02-11,WGR,Energy Minerals,0.947634
2004-02-11,CWT,Utilities,0.934181
2004-02-11,BLL,Process Industries,0.922862


In [8]:
print('Shape of raw dataset: {}'.format(df.shape))

Shape of raw dataset: (37360, 3)


In [25]:
print('Number of unique dates: {}'.format(df.index.nunique()))

Number of unique dates: 467


In [20]:
# sprawdzenie czy występują braki danych
df.isnull().sum()

symbol    0
sector    0
score     0
dtype: int64

In [21]:
# sprawdzenie czy występują duplikaty
df.duplicated().sum()

0

In [22]:
# sprawdzenie typów danych
df.dtypes

symbol     object
sector     object
score     float64
dtype: object

In [24]:
df.symbol.unique()

array(['SU', 'GGG', 'WGR', ..., 'DELL', 'BOOT', 'AGCO'], dtype=object)

In [26]:
print('Number of unique symbols: {}'.format(df.symbol.nunique()))

Number of unique symbols: 1834


In [27]:
df.sector.unique()

array(['Energy Minerals', 'Producer Manufacturing', 'Utilities',
       'Process Industries', 'Consumer Services', 'Transportation',
       'Retail Trade', 'Finance', 'Health Technology', 'Miscellaneous',
       'Non-Energy Minerals', 'Distribution Services',
       'Consumer Non-Durables', 'Commercial Services',
       'Technology Services', 'Consumer Durables', 'Health Services',
       'Electronic Technology', 'Industrial Services', 'Communications'],
      dtype=object)

In [28]:
df.score.unique()

array([0.95372705, 0.9527531 , 0.94763414, ..., 0.69795422, 0.69774066,
       0.69564356])

In [29]:
# usuniecie zbednej czesci w kolumnie symbol
df['symbol'] = df['symbol'].str.replace(".", " ")
df['symbol'] = df['symbol'].str.split(' ')

xyz = []
for x in df["symbol"].to_numpy():
  xyz.append(x[0])
df["symbol"] = xyz

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


### Pobieranie danych z Yahoo

In [30]:
df2 = df.sort_values(["symbol", "date"])

In [32]:
tickers = df2.symbol.unique()
list_tickers = tickers.tolist()
Symbol = yf.Tickers(list_tickers)

In [33]:
data = yf.download(list_tickers, start='2004-02-10', end='2022-02-10', interval="1d")['Close']

[*********************100%***********************]  1804 of 1804 completed

393 Failed downloads:
- ARG: No data found for this date range, symbol may be delisted
- DFODQ: No data found, symbol may be delisted
- POT: No data found for this date range, symbol may be delisted
- AACB: No data found for this date range, symbol may be delisted
- KFN: No data found for this date range, symbol may be delisted
- UTIW: No data found for this date range, symbol may be delisted
- ASNAQ: No data found, symbol may be delisted
- HOFD: No data found, symbol may be delisted
- SIAL: No data found for this date range, symbol may be delisted
- BCR: No data found for this date range, symbol may be delisted
- SPSS: No data found for this date range, symbol may be delisted
- BLKIA: No data found for this date range, symbol may be delisted
- WNR: No data found for this date range, symbol may be delisted
- LNY: No data found for this date range, symbol may be delisted
- BRSS: No data found, symbol may be deli

In [34]:
data.head(5)

,A,AACB,AAIC,AAP,AAPL,AAT,AAWW,ABBV,ABC,ABCD,...,XTO,XYL,YELL,YLWDF,YUM,ZBRA,ZD,ZLC,ZQKSQ,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2004-02-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-02-10 00:00:00,26.731045,NaN,500.600006,27.280001,0.410357,NaN,NaN,NaN,14.1675,NaN,...,NaN,NaN,239025.0,NaN,11.926671,43.740002,9.769565,NaN,NaN,NaN
2004-02-11 00:00:00,26.752504,NaN,514.799988,27.753332,0.425000,NaN,NaN,NaN,14.2500,NaN,...,NaN,NaN,244200.0,NaN,12.117182,47.119999,10.500000,NaN,NaN,NaN
2004-02-12 00:00:00,26.409157,NaN,515.400024,28.046667,0.423750,NaN,NaN,NaN,14.1450,NaN,...,NaN,NaN,247500.0,NaN,12.677930,46.866669,10.326087,NaN,NaN,NaN
2004-02-13 00:00:00,26.523605,NaN,512.000000,27.933332,0.410714,NaN,NaN,NaN,14.1525,NaN,...,NaN,NaN,245775.0,NaN,12.692308,46.453335,10.186957,NaN,NaN,NaN


In [35]:
print('Shape of dataset from Yahoo: {}'.format(data.shape))

Shape of dataset from Yahoo: (4551, 1804)


In [88]:
# puste kolumny
columns_nan = data.columns[data.isna().all()].tolist()
len(columns_nan)

In [36]:
data = data.dropna(how='any', axis=1, thresh=1)

In [38]:
print('Shape of dataset from Yahoo without empty columns: {}'.format(data.shape))

Shape of dataset from Yahoo without empty columns: (4551, 1411)


In [40]:
data2 = pd.melt(data, id_vars='date', value_vars=data.columns.to_list())
data2

,date,variable,value
0,2004-02-09,A,NaN
1,2004-02-10,A,26.731045
2,2004-02-11,A,26.752504
3,2004-02-12,A,26.409157
4,2004-02-13,A,26.523605
...,...,...,...
6421456,2022-02-03,ZTS,200.919998
6421457,2022-02-04,ZTS,199.539993
6421458,2022-02-07,ZTS,200.320007
6421459,2022-02-08,ZTS,201.300003


In [45]:
#usuwanie pustych wartosci
data3 = data2.dropna()
#utworzenie pustej kolumny
data3["stopa_zwrotu"] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
df_rr = pd.DataFrame(columns=['date', 'variable', 'value','stopa_zwrotu'])

In [48]:
# 47 minut
# stopa zwrotu dla danych dziennych
symbols = data2["variable"].unique().tolist()

for sym in symbols:

    data_symbol = data3.loc[data3["variable"] == sym]

    for i in range(0, len(data_symbol)):
        if i+1<len(data_symbol):
            data_symbol["stopa_zwrotu"].iloc[i+1] = (data_symbol["value"].iloc[i+1]/data_symbol["value"].iloc[i])-1 
    
    df_rr = pd.concat([df_rr, data_symbol])

df_rr


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


,date,variable,value,stopa_zwrotu
1,2004-02-10,A,26.731045,NaN
2,2004-02-11,A,26.752504,0.000803
3,2004-02-12,A,26.409157,-0.012834
4,2004-02-13,A,26.523605,0.004334
5,2004-02-17,A,26.816881,0.011057
...,...,...,...,...
6421456,2022-02-03,ZTS,200.919998,-0.006183
6421457,2022-02-04,ZTS,199.539993,-0.006868
6421458,2022-02-07,ZTS,200.320007,0.003909
6421459,2022-02-08,ZTS,201.300003,0.004892


In [57]:
# funkcja działa
df_rr.loc[df_rr['variable'] == "SU"]

,date,variable,value,stopa_zwrotu
5429344,2004-02-10,SU,13.175000,NaN
5429345,2004-02-11,SU,13.285000,0.008349
5429346,2004-02-12,SU,12.960000,-0.024464
5429347,2004-02-13,SU,12.830000,-0.010031
5429348,2004-02-17,SU,12.985000,0.012081
...,...,...,...,...
5433889,2022-02-03,SU,29.219999,-0.038816
5433890,2022-02-04,SU,28.719999,-0.017112
5433891,2022-02-07,SU,28.990000,0.009401
5433892,2022-02-08,SU,28.469999,-0.017937


In [71]:
data4 = pd.merge(df, df_rr,  how='left', left_on=['date','symbol'], right_on = ['date','variable'])

In [72]:
data4.head(5)

,date,symbol,sector,score,variable,value,stopa_zwrotu
0,2004-02-11,SU,Energy Minerals,0.953727,SU,13.285000,0.008349
1,2004-02-11,GGG,Producer Manufacturing,0.952753,GGG,9.388889,0.011734
2,2004-02-11,WGR,Energy Minerals,0.947634,NaN,NaN,NaN
3,2004-02-11,CWT,Utilities,0.934181,CWT,14.720000,0.004778
4,2004-02-11,BLL,Process Industries,0.922862,BLL,8.095000,0.002787


In [78]:
dataset = data4.dropna(axis = 0, how ='any')

In [74]:
dataset.head(5)

,date,symbol,sector,score,variable,value,stopa_zwrotu
0,2004-02-11,SU,Energy Minerals,0.953727,SU,13.285000,0.008349
1,2004-02-11,GGG,Producer Manufacturing,0.952753,GGG,9.388889,0.011734
3,2004-02-11,CWT,Utilities,0.934181,CWT,14.720000,0.004778
4,2004-02-11,BLL,Process Industries,0.922862,BLL,8.095000,0.002787
5,2004-02-11,APA,Energy Minerals,0.912117,APA,39.830002,0.005808


In [80]:
dataset = dataset.drop('variable', axis=1)

In [81]:
dataset = dataset.rename(columns={'value': 'close', 'stopa_zwrotu': 'return_rate'})

In [82]:
print("Old data frame length:", len(data4), "\nNew data frame length:", 
       len(dataset), "\nNumber of rows with at least 1 NA value: ",
       (len(data4)-len(dataset)))

Old data frame length: 37360 
New data frame length: 30508 
Number of rows with at least 1 NA value:  6852


In [84]:
dataset.head(5)

,date,symbol,sector,score,close,return_rate
0,2004-02-11,SU,Energy Minerals,0.953727,13.285000,0.008349
1,2004-02-11,GGG,Producer Manufacturing,0.952753,9.388889,0.011734
3,2004-02-11,CWT,Utilities,0.934181,14.720000,0.004778
4,2004-02-11,BLL,Process Industries,0.922862,8.095000,0.002787
5,2004-02-11,APA,Energy Minerals,0.912117,39.830002,0.005808


In [86]:
from google.colab import files

dataset.to_csv('CONVICTIONSUMMARY.csv', index=False) 
files.download('CONVICTIONSUMMARY.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>